In [26]:

# play sentences in Anki
# initial date: 2015-11-20

# to-do: add image field? "crock" in "chili served in a crock"


import pymongo
from pymongo import MongoClient

# codecs if for read and write files with unicode, encoding 'utf-8'
import codecs
import re

#connect to MongoDB
client = MongoClient('localhost', 27017)
db = client.dpdb  # databas: dpdb
geweiDict = db.geweiDict201511 # collection (table): geweiDict201511
#geweiDict = db.geweiDictTest


fdict = open('anki\\sPlayAnki.txt', 'w')
#fdict = codecs.open('anki\\sentPlayAnki.txt', 'w', 'utf-8')
    
results = geweiDict.find({}, {"_id":0, "hw":1, "infl":1, "subhw":1, "prn":1, "alles":1, 
                              "idpvs":1, "sublist":1, "audiofile":1, "alles":1, "fr":1})
# sort method will use much more memory
#results = geweiDict.find({}, {"_id":0, "hw":1, "prn":1, "alles":1, "idpvs":1, "sublist":1}).sort("hw", 1).limit(1000)

# total number of head words
totalhw = 0
# number of head words without example sentences
cnt_noes = 0

# Produce Anki note fields and tags:
for result in results:
    
    hw = result.get("hw")    
    if hw:
        word = hw.encode('ascii', 'ignore')
        #word = hw.encode('utf-8')
        print word
        totalhw += 1
    else:
        print "no head word"
        continue

    
    # get items in "infl" and "subhw"
    infls = result.get("infl")
    subhws = result.get("subhw")   
    
    homograph = 0
    audiofile = result.get("audiofile")
    
    idpvs = result.get("idpvs")
    sublists = result.get("sublist") #sublists is a list whose items are dictionary
    
    fr = result.get("fr")

    alles = result.get("alles")        
    if alles: totales = len(alles)
    else: totales=0
        
    if totales >= 100: nes = "100+"
    elif totales >= 50: nes = "50-99"
    elif totales >= 10: nes = "10-49"
    elif totales >=  3: nes = "3-9"
    elif totales >=  2: nes = "2"
    elif totales >=  1: nes = "1"
    else: nes = "0"
        
    
    if sublists:
        cnt = 1
        idiom = 0
        
        # each "es" or "idpv" of sublist in sublists will be an Anki note
        for sublist in sublists: #sublist is dictionary

            # Anki note field: PlaySent
            if sublist.has_key("es"):
                # has at least one example sentence, but this es is empty
                if len(sublist["es"]) == 0 and alles:
                    print word, ": empyt es"
                    continue
                # no example sentence, write head word to PlaySent field
                elif len(sublist["es"]) == 0 and not alles:
                    cnt_noes += 1
                    fdict.write(word)
                else:
                    for es in sublist["es"]:                    
                        #sPlay = es.encode('ascii', 'ignore').replace(word, "___")
                        sPlay = es.encode('ascii', 'ignore')
                        insensitive_replace = re.compile(re.escape(word), re.IGNORECASE)
                        sPlay = insensitive_replace.sub('___', sPlay)
                        
                        if infls:
                            for infl in infls:
                                sPlay = sPlay.replace(infl, "...")
                        if subhws:
                            for subhw in subhws:
                                sPlay = sPlay.replace(subhw, "???")
                        fdict.write(str(cnt) + '<font color="blue">-es: </font>' + sPlay + "<br>")
                    
            if sublist.has_key("idpv"):
                if len(sublist["idpv"]) == 0:
                    print word, ": empyt idpv"
                    continue
                #sPlay = sublist["idpv"].encode('ascii', 'ignore').replace(word, "___")
                sPlay = sublist["idpv"].encode('ascii', 'ignore')
                insensitive_replace = re.compile(re.escape(word), re.IGNORECASE)
                sPlay = insensitive_replace.sub('___', sPlay)
                if sPlay.find(":") >= 0: idiom = 1
                
                if infls:
                    for infl in infls:
                        sPlay = sPlay.replace(infl, "...")
                if subhws:
                    for subhw in subhws:
                        sPlay = sPlay.replace(subhw, "???")
                fdict.write('<font color="blue">*idpv*</font> ' + sPlay + "<br>")
            if cnt%1 == 0: fdict.write('<br>')
            fdict.write(" \t")            
            
            
            # Anki note field: head word
            fdict.write(word + "\t")           
            
            # Anki note field: Pronunciation in ASCII IPA symbols
            prns = result.get("prn")
            if prns:
                for prn in prns:
                    fdict.write("/" + prn + "/<br>")
            fdict.write(" \t")  # add one space before tab to separate tabs in case a field is empty. 
            

            # Anki note field: Audio    
            if word: 
                if audiofile: 
                    fdict.write("[sound:" + audiofile + "]")
                    homograph = 1
                else: fdict.write("[sound:" + word + ".wav]")
            if subhws:
                for subhw in subhws:
                    fdict.write("<br>[sound:" + subhw.encode('ascii', 'ignore') + ".wav]")
            fdict.write(" \t")    


            # Anki note field: definition
            if sublist.has_key("def"):                
                fdict.write(str(cnt) + '-def: ' + sublist["def"].encode('ascii', 'ignore') + "<br><br>")            
            fdict.write(" \t")
                
            # Anki note field: Wow
            if sublist.has_key("es"):  
                for es in sublist["es"]:
                    fdict.write(str(cnt) + '<font color="red">-es: </font>' + 
                                es.encode('ascii', 'ignore') + "<br>")                
            if sublist.has_key("idpv"): 
                fdict.write('<font color="red">*idpv*</font> ' + 
                            sublist["idpv"].encode('ascii', 'ignore') + "<br>")
            if cnt%1 == 0: fdict.write('<br>') 
            fdict.write(" \t")
            
               
            tags = "splay"
            if idiom == 1: tags += " idiom"
            if fr: tags += " fr" # Foreign word (French, Mexican Spanish)       
            if homograph == 1 and word != 'con' : tags += " homog" # homog -- homograph
            tags += " es" + nes # level of a word by the number of examples
            
            # Anki note field: Footnote            
            fdict.write(tags + "\t")
            

            # Anki tags
            fdict.write(tags + "\n")
            
            cnt += 1
    
fdict.close()
print "total head words: ", totalhw, ". Number of head words without example sentences: ", cnt_noes





zip file
zydeco
WW
www
YA
yd.
yeshiva
yesteryear
yeti
yew
yew : empyt es
Yiddish
Yiddish : empyt es
yikes
yip
yippee
yo-yo
yo
yob
yodel
yoga
yoga : empyt es
yogurt
yoke
yokel
Yom Kippur
yon
yonks
yoo-hoo
yoof
yore
you'd
you'll
you're
you-all
you-know-who
youngish
youngster
young gun
young offender
youth hostel
yr.
YT
yuan
yucca
yuck
Yule
Yuletide
yule log
yummy
yup
yuppie
yuppify
Y chromosome
zaftig
zany
zapper
zapper : empyt es
zebra
zed
zee
zeitgeist
Zen
zephyr
zero-sum game
zero gravity
zero hour
zilch
zillion
Zimmer frame
zinc
zine
zing
zing : empyt es
zinger
zinnia
Zionism
Zionism : empyt es
zip
zip
zip
zip : empyt es
zip
zippo
zippy
zippy : empyt es
zip code
zit
zither
zodiac
zombie
zonal
zoned out
zone defense
zoning
zonked
zoo
zookeeper
zoological park
zoology
zoom
zoom lens
zoot suit
Zoroastrianism
zucchini
Zulu
zwieback
zygote
con
con
con
con
woolly
woozy
wop
word-perfect
wording
wordless
wordplay
word processing
work-shy
work-study program
work-to-rule
workaholic
workbench
w

In [6]:
word="abc\nu0x99999"
print word
print word.encode('ascii', 'ignore')

abc
u0x99999
abc
u0x99999


In [2]:
tstr="we adjust oursleves to the weather"
print tstr.replace("adjust", "___")

we ___ oursleves to the weather


In [25]:
sPlay = "You mean he lied to us? Bingo!"
repl = "bingo"
resub = re.compile(re.escape(repl), re.IGNORECASE)
print resub.sub("___", sPlay) # sPlay doesn't change
print sPlay

You mean he lied to us? ___!
You mean he lied to us? Bingo!
